In [2]:
%load_ext autoreload
%autoreload 2

import os
import cv2
import matplotlib.pyplot as plt
from attention.img_proc.img_process import *
from attention.img_proc.img_split import *
from attention.models.face_models import *
from attention.utils.img_plot import *
import copy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


SyntaxError: invalid syntax (img_process.py, line 44)

In [18]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [27]:
image_path = os.path.join(os.path.expanduser('~'), "Desktop", "origin")
output_path = os.path.join(os.path.expanduser('~'), "Desktop", "origin-face-result")

In [28]:
#return a list containing all the photos in array of origin folder that we access thanks to the path 
list_images_origin =load_images_from_folder(image_path)
len(list_images_origin)

67

In [23]:
type(list_images_origin[0])

numpy.ndarray

In [30]:
for idx, photo in enumerate(list_images_origin):
    image_name = f'frame_box_{idx}.png'
    image_array = photo
    image = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
    n_split_w = 2
    n_split_h = 2
    crops = split_image(image, n_split_w, n_split_h, 0.02, 0.02)
    # show_crops(crops)
    
    bbox_crop_list = []
    bbox_crop_list_origin = []
    for crop in crops:
        coord_set = detect_face(crop.image)
        bbox_crop_list.append(coord_set)
        coord_set_origin = reconstruct_coord(crop, coord_set)
        bbox_crop_list_origin.append(coord_set_origin)
        
    crops_annotated = []
    for i in range(len(crops)):
        crop_annotated = copy.copy(crops[i])
        crop_annotated.image = output_image_bboxes(crop_annotated.image, bbox_crop_list[i])
        crops_annotated.append(crop_annotated)
    # show_crops(crops_annotated)
    
    bbox_list = bbox_merge(bbox_crop_list_origin)
    image_output = output_image_bboxes(image, bbox_list)
    image_output_rgb = cv2.cvtColor(image_output, cv2.COLOR_RGB2BGR)
    cv2.imwrite(os.path.join(output_path,image_name), image_output_rgb)
